## Parte 1: Carregamento e Limpeza de Dados

Para começar, carregamos nossa principal ferramenta de trabalho, a biblioteca pandas, e em seguida, importamos as 4 planilhas de dados (partidas, estatísticas, cartões e gols).

In [2]:
import pandas as pd

In [3]:
try:
    df_full = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-full.csv')
    df_stats = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-estatisticas-full.csv')
    df_cartoes = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-cartoes.csv')
    df_gols = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-gols.csv')

    print("Os 4 arquivos CSV foram carregados com sucesso!")
    
except FileNotFoundError as e:
    print(f"Erro: O arquivo {e.filename} não foi encontrado.")
    print("Por favor, certifique-se de que os arquivos CSV estão no mesmo diretório que o seu notebook.")

Os 4 arquivos CSV foram carregados com sucesso!


Ajuste de Data e Hora, além de renomear a coluna ID em df_full para 'partida_id', padronizando

In [4]:
# Ajustando os tipos de dados de data e hora
df_full['data'] = pd.to_datetime(df_full['data'], format='%d/%m/%Y', errors='coerce')
df_full['hora'] = pd.to_datetime(df_full['hora'], format='%H:%M', errors='coerce').dt.time

# Renomeando a coluna 'ID' para 'partida_id' para padronização
df_full.rename(columns={'ID': 'partida_id'}, inplace=True)

print("Formatos de data/hora e coluna de ID ajustados com sucesso.")

Formatos de data/hora e coluna de ID ajustados com sucesso.


#### Sincronizando as Tabelas, garantindo qualidade

Para garantir a máxima confiabilidade, vamos trabalhar apenas com partidas que possuem registros completos em todas as quatro tabelas. Isso evita análises baseadas em dados incompletos.

In [5]:
# Encontrando os IDs de partida comuns a todas as tabelas
ids_full = set(df_full['partida_id'])
ids_stats = set(df_stats['partida_id'])
ids_cartoes = set(df_cartoes['partida_id'])
ids_gols = set(df_gols['partida_id'])

# Encontra a interseção de IDs
common_ids = set.intersection(ids_full, ids_stats, ids_cartoes, ids_gols)

print(f"Número de partidas em comum a todas as 4 bases: {len(common_ids)}")

Número de partidas em comum a todas as 4 bases: 3761


Agora, filtramos nossas tabelas para manter apenas essas partidas "validadas".

In [6]:
# Filtrando os DataFrames para manter apenas os dados completos
common_ids_list = list(common_ids)

df_full_cleaned = df_full[df_full['partida_id'].isin(common_ids_list)].copy()
df_stats_cleaned = df_stats[df_stats['partida_id'].isin(common_ids_list)].copy()
df_cartoes_cleaned = df_cartoes[df_cartoes['partida_id'].isin(common_ids_list)].copy()
df_gols_cleaned = df_gols[df_gols['partida_id'].isin(common_ids_list)].copy()

print("Filtro aplicado. Comparativo do número de linhas (Antes -> Depois):")
print(f"Partidas:      {len(df_full)} -> {len(df_full_cleaned)}")
print(f"Estatísticas:  {len(df_stats)} -> {len(df_stats_cleaned)}")
print(f"Cartões:       {len(df_cartoes)} -> {len(df_cartoes_cleaned)}")
print(f"Gols:          {len(df_gols)} -> {len(df_gols_cleaned)}")

Filtro aplicado. Comparativo do número de linhas (Antes -> Depois):
Partidas:      8785 -> 3761
Estatísticas:  17570 -> 7522
Cartões:       20953 -> 19254
Gols:          9861 -> 9711


#### Tratando Campos Vazios

O último passo da limpeza é preencher campos vazios (NaN) de forma inteligente para evitar erros e inconsistências. Aqui, não apagamos os dados NaN pois são grande parte do nosso banco de dados e dados como 'posse de bola', serão usados em análises futuras. Sobre as formações dos times, como são poucos dados e não pretendo usar, 'dropei' essas colunas.

In [8]:
# Removendo as colunas de formação do DataFrame df_full_cleaned
df_full_cleaned.drop(columns=['formacao_mandante', 'formacao_visitante'], inplace=True, errors='ignore')
# Em df_stats, estatísticas ausentes viram 'Sem Info' (serão tratadas depois
df_stats_cleaned['posse_de_bola'].fillna('Sem Info', inplace=True)
df_stats_cleaned['precisao_passes'].fillna('Sem Info', inplace=True)

# Em df_cartoes, removemos a coluna 'num_camisa' e preenchemos 'posicao'
df_cartoes_cleaned.drop(columns=['num_camisa'], inplace=True, errors='ignore')
df_cartoes_cleaned['posicao'].fillna('Sem Info', inplace=True)

# Em df_gols, um gol sem tipo definido é assumido como 'Gol Normal'
df_gols_cleaned['tipo_de_gol'].fillna('Gol Normal', inplace=True)

print("Campos vazios tratados com sucesso.")

Campos vazios tratados com sucesso.


C:\Users\rossi\AppData\Local\Temp\ipykernel_2460\3709112189.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_stats_cleaned['posse_de_bola'].fillna('Sem Info', inplace=True)
C:\Users\rossi\AppData\Local\Temp\ipykernel_2460\3709112189.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

## Parte 2: Construindo a Tabela Mestra  (EM MANUTENÇÃO)
Com os dados limpos e sincronizados, o objetivo agora é criar uma única tabela onde cada linha representa uma partida e contém as estatísticas do time da casa e do visitante, lado a lado

#### Separando Estatísticas: Mandante vs. Visitante

Primeiro, separamos os dados da tabela de estatísticas em duas novas tabelas: uma só para os times da casa (mandantes) e outra só para os visitantes.

In [9]:
# # Separando as estatísticas
# stats_com_times = pd.merge(df_stats_cleaned, df_full_cleaned[['partida_id', 'mandante', 'visitante']], on='partida_id')
# mandante_stats = stats_com_times[stats_com_times['clube'] == stats_com_times['mandante']].copy()
# visitante_stats = stats_com_times[stats_com_times['clube'] == stats_com_times['visitante']].copy()

# print("Estatísticas separadas para mandantes e visitantes.")

Estatísticas separadas para mandantes e visitantes.


In [11]:
# mandante_stats.head(3)

,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios,mandante,visitante
0,4607,1,Fluminense,0,0,Sem Info,0,Sem Info,0,0,0,0,0,Fluminense,Figueirense
3,4608,1,Internacional,0,0,Sem Info,0,Sem Info,0,0,0,0,0,Internacional,Vitoria
4,4612,1,Bahia,0,0,Sem Info,0,Sem Info,0,0,0,0,0,Bahia,Cruzeiro


In [12]:
# visitante_stats.head(3)

,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios,mandante,visitante
1,4607,1,Figueirense,0,0,Sem Info,0,Sem Info,0,0,0,0,0,Fluminense,Figueirense
2,4608,1,Vitoria,0,0,Sem Info,0,Sem Info,0,0,0,0,0,Internacional,Vitoria
5,4612,1,Cruzeiro,0,0,Sem Info,0,Sem Info,0,0,0,0,0,Bahia,Cruzeiro


Para que na tabela final saibamos exatamente a qual time cada estatística pertence, adicionamos os prefixos mandante_ e visitante_ às colunas.

In [14]:
# # Adicionando prefixos
# colunas_stats = [
#     'chutes', 'chutes_no_alvo', 'posse_de_bola', 'passes', 'precisao_passes',
#     'faltas', 'cartao_amarelo', 'cartao_vermelho', 'impedimentos', 'escanteios'
# ]

# # Prefixando e selecionando colunas para a tabela de mandantes
# mandante_stats_renamed = mandante_stats[colunas_stats].add_prefix('mandante_')
# mandante_stats_renamed['partida_id'] = mandante_stats['partida_id']

# # Prefixando e selecionando colunas para a tabela de visitantes
# visitante_stats_renamed = visitante_stats[colunas_stats].add_prefix('visitante_')
# visitante_stats_renamed['partida_id'] = visitante_stats['partida_id']

# print("Prefixos adicionados com sucesso!")
# display(mandante_stats_renamed.head(300))

Prefixos adicionados com sucesso!


,mandante_chutes,mandante_chutes_no_alvo,mandante_posse_de_bola,mandante_passes,mandante_precisao_passes,mandante_faltas,mandante_cartao_amarelo,mandante_cartao_vermelho,mandante_impedimentos,mandante_escanteios,partida_id
0,0,0,Sem Info,0,Sem Info,0,0,0,0,0,4607
3,0,0,Sem Info,0,Sem Info,0,0,0,0,0,4608
4,0,0,Sem Info,0,Sem Info,0,0,0,0,0,4612
7,0,0,Sem Info,0,Sem Info,0,0,0,0,0,4611
9,0,0,Sem Info,0,Sem Info,0,0,0,0,0,4610
...,...,...,...,...,...,...,...,...,...,...,...
591,0,0,Sem Info,0,Sem Info,0,0,0,0,0,4948
593,0,0,Sem Info,0,Sem Info,0,0,0,0,0,4950
594,0,0,Sem Info,0,Sem Info,0,0,0,0,0,4951
596,0,0,Sem Info,0,Sem Info,0,0,0,0,0,4953
